## Project Description

This project focuses on predicting customer churn for a telecommunications provider using multiple data sources, including contract details, personal information, internet services, and phone services. The goal is to identify customers who are likely to leave and understand the key factors influencing churn.

The analysis includes data preprocessing, feature engineering, exploratory data analysis, and training multiple machine learning models using scikit-learn pipelines. Model performance is evaluated using classification metrics to select the most effective approach.


In [60]:
# Core
import warnings
from datetime import datetime

import numpy as np
import pandas as pd

# Modeling
from sklearn.base import clone
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier

# Metrics
from sklearn.metrics import roc_auc_score, accuracy_score, precision_recall_fscore_support
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)

## Data Loading

In [61]:
contract = pd.read_csv("contract.csv")
personal = pd.read_csv("personal.csv")
internet = pd.read_csv("internet.csv")
phone = pd.read_csv("phone.csv")

In [62]:
print('Shapes:', contract.shape, personal.shape, internet.shape, phone.shape)
display(contract.head(5))
display(personal.head(5))
display(internet.head(5))
display(phone.head(5))

Shapes: (7043, 8) (7043, 5) (5517, 8) (6361, 2)


,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65


,customerID,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No


,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,Fiber optic,No,No,No,No,No,No


,customerID,MultipleLines
0,5575-GNVDE,No
1,3668-QPYBK,No
2,9237-HQITU,No
3,9305-CDSKC,Yes
4,1452-KIOVK,Yes


## Merge, Target, and Basic Sanity Checks

In [63]:
# Merge on customerID
dfs = [contract, personal, internet, phone]
for df in dfs:
    if 'customerID' not in df.columns:
        raise ValueError('Expected key column `customerID` is missing in one of the tables.')

data = (contract.merge(personal, on='customerID', how='left').merge(internet, on='customerID', how='left').merge(phone, on='customerID', how='left'))
print('Merged shape:', data.shape)

if 'EndDate' not in data.columns:raise ValueError('Expected column `EndDate` not found.')

data['churn'] = (data['EndDate'] != 'No').astype(int)

# Quick churn rate sanity check
churn_rate = data['churn'].mean()
print(f'Churn rate: {churn_rate:.4f} ({100*churn_rate:.1f}%)')

Merged shape: (7043, 20)
Churn rate: 0.2654 (26.5%)


## Feature Engineering (Tenure) and Leakage Control

In [64]:
# Engineer tenure (months) from BeginDate up to snapshot date
SNAPSHOT = datetime(2020, 2, 1)

def to_dt(s):
    return pd.to_datetime(s, errors='coerce')

if 'BeginDate' not in data.columns:
    raise ValueError('Expected column `BeginDate` not found.')

data['BeginDate_dt'] = to_dt(data['BeginDate'])
data['tenure_months'] = ((SNAPSHOT - data['BeginDate_dt']).dt.days / 30.44).clip(lower=0)
data['tenure_months'] = data['tenure_months'].fillna(data['tenure_months'].median())

# Leakage control: drop columns that reveal post-snapshot info or direct target
leak_cols = ['EndDate']
drop_cols = ['BeginDate_dt']

data = data.drop(columns=drop_cols)
print('Columns after engineering & dropping leakage helpers:', data.columns.tolist()[:20])

Columns after engineering & dropping leakage helpers: ['customerID', 'BeginDate', 'EndDate', 'Type', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines']


## Feature Sets

In [65]:
# Identify categorical vs numeric features
target = 'churn'
RANDOM_STATE = 42 

numeric_cols = []
categorical_cols = []

for col in data.columns:
    if col == target:
        continue
    if col in leak_cols:
        continue
    if pd.api.types.is_numeric_dtype(data[col]):
        numeric_cols.append(col)
    else:
        categorical_cols.append(col)

print('Numeric columns (sample):', numeric_cols[:10])
print('Categorical columns (sample):', categorical_cols[:10])

Numeric columns (sample): ['MonthlyCharges', 'SeniorCitizen', 'tenure_months']
Categorical columns (sample): ['customerID', 'BeginDate', 'Type', 'PaperlessBilling', 'PaymentMethod', 'TotalCharges', 'gender', 'Partner', 'Dependents', 'InternetService']


## Stratified 60/20/20 Split

In [66]:
X = data.drop(columns=[target] + leak_cols)
y = data[target].copy()

# 60/20/20: first split off test (20%), then split remaining into train/val (75/25 of remaining → 60/20 overall)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=RANDOM_STATE)

print('Train/Val/Test sizes:', X_train.shape, X_val.shape, X_test.shape)
print('Churn rate (train/val/test):', y_train.mean().round(5), y_val.mean().round(4), y_test.mean().round(5))

Train/Val/Test sizes: (4225, 20) (1409, 20) (1409, 20)
Churn rate (train/val/test): 0.26533 0.2654 0.26544


## Preprocessing Pipeline

In [67]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler(with_mean=False))])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=True))])

preprocess = ColumnTransformer(
    transformers=[('num', numeric_transformer, numeric_cols), ('cat', categorical_transformer, categorical_cols),], remainder='drop', sparse_threshold=0.3)

In [68]:
pipe_lr_raw = Pipeline(steps=[
    ('preprocess', preprocess),
    ('clf', LogisticRegression(
        max_iter=1000,
        solver='liblinear',
        random_state=42))])

pipe_lr_bal = Pipeline(steps=[
    ('preprocess', preprocess),
    ('clf', LogisticRegression(
        max_iter=1000,
        solver='liblinear',
        class_weight='balanced',
        random_state=42))])

In [69]:
print("raw solver:", pipe_lr_raw.named_steps['clf'].get_params()['solver'])
print("bal solver:", pipe_lr_bal.named_steps['clf'].get_params()['solver'])

raw solver: liblinear
bal solver: liblinear


## Baseline Models (LogReg) - Raw vs Balanced

In [70]:
warnings.filterwarnings("ignore", category=ConvergenceWarning)

def eval_model(model, X_tr, y_tr, X_va, y_va, name='model'):
    model.fit(X_tr, y_tr)
    proba = model.predict_proba(X_va)[:, 1] if hasattr(model, 'predict_proba') else model.decision_function(X_va)
    auc = roc_auc_score(y_va, proba)
    preds = (proba >= 0.5).astype(int)
    acc = accuracy_score(y_va, preds)
    print(f'{name}: AUC={auc:.4f} | ACC={acc:.4f}')
    return auc, acc, proba

def make_lr_pipeline(balanced=False):
    clf = LogisticRegression(solver='saga', class_weight='balanced' if balanced else None, max_iter=5000, tol=1e-3, n_jobs=-1, random_state=RANDOM_STATE)
    return Pipeline(steps=[('prep', preprocess), ('clf', clf)])

pipe_lr_raw = make_lr_pipeline(balanced=False)
pipe_lr_bal = make_lr_pipeline(balanced=True)

print("\nEvaluating baseline Logistic Regression...")
auc_lr_raw, acc_lr_raw, _ = eval_model(pipe_lr_raw, X_train, y_train, X_val, y_val, "LR raw")
auc_lr_bal, acc_lr_bal, _ = eval_model(pipe_lr_bal, X_train, y_train, X_val, y_val, "LR balanced")


Evaluating baseline Logistic Regression...
LR raw: AUC=0.8430 | ACC=0.8013
LR balanced: AUC=0.8436 | ACC=0.7743


## Model Tuning (Small Grids) - RF / GBDT / HGB

In [71]:
print("Running Fast model searches (RF / HGB):", flush=True)

# Random Forest
pipe_rf = Pipeline(steps=[
    ('prep', preprocess),
    ('clf', RandomForestClassifier(random_state=RANDOM_STATE))])

param_rf = {
    'clf__n_estimators': [200, 400, 600],
    'clf__max_depth': [None, 8, 14],
    'clf__min_samples_split': [2, 10, 20],
    'clf__min_samples_leaf': [1, 5, 10],
    'clf__class_weight': [None, 'balanced']}

rs_rf = RandomizedSearchCV(
    pipe_rf,
    param_distributions=param_rf,
    n_iter=10,           
    scoring='roc_auc',
    cv=3,
    n_jobs=-1,
    verbose=1,
    random_state=RANDOM_STATE,
    refit=True)

print("\n[Fitting] RandomForest randomized search:", flush=True)
rs_rf.fit(X_train, y_train)
print(f"RF best AUC (CV): {rs_rf.best_score_:.4f} | params: {rs_rf.best_params_}", flush=True)


# HistGradientBoosting
to_dense = FunctionTransformer(lambda X: X.toarray() if hasattr(X, "toarray") else X)

pipe_hgb = Pipeline(steps=[
    ('prep', preprocess),
    ('to_dense', to_dense),
    ('clf', HistGradientBoostingClassifier(
        random_state=RANDOM_STATE,
        early_stopping=True,
        validation_fraction=0.1,
        n_iter_no_change=10))])

param_hgb = {
    'clf__learning_rate': [0.03, 0.05, 0.1],
    'clf__max_depth': [None, 6, 10],
    'clf__max_iter': [100, 200]}

rs_hgb = RandomizedSearchCV(
    pipe_hgb,
    param_distributions=param_hgb,
    n_iter=8,             
    scoring='roc_auc',
    cv=3,
    n_jobs=-1,
    verbose=1,
    random_state=RANDOM_STATE,
    refit=True)

print("\n[Fitting] HistGradientBoosting randomized search:", flush=True)
rs_hgb.fit(X_train, y_train)
print(f"HGB best AUC (CV): {rs_hgb.best_score_:.4f} | params: {rs_hgb.best_params_}", flush=True)

# Compare on the same validation split
candidates = [
    ('LR raw', pipe_lr_raw),
    ('LR balanced', pipe_lr_bal),
    ('RF (best)', rs_rf.best_estimator_),
    ('HGB (best)', rs_hgb.best_estimator_),]

print("\n[Comparison] Scoring candidates on validation split:", flush=True)
rows = []
for name, est in candidates:
    est.fit(X_train, y_train)
    proba = est.predict_proba(X_val)[:, 1] if hasattr(est, 'predict_proba') else est.decision_function(X_val)
    auc = roc_auc_score(y_val, proba)
    preds = (proba >= 0.5).astype(int)
    acc = accuracy_score(y_val, preds)
    rows.append({'Model': name, 'Val AUC': auc, 'Val ACC': acc})

results_df = pd.DataFrame(rows).sort_values('Val AUC', ascending=False).reset_index(drop=True)
print(results_df.to_string(index=False), flush=True)

Running Fast model searches (RF / HGB):

[Fitting] RandomForest randomized search:
Fitting 3 folds for each of 10 candidates, totalling 30 fits
RF best AUC (CV): 0.8454 | params: {'clf__n_estimators': 400, 'clf__min_samples_split': 2, 'clf__min_samples_leaf': 1, 'clf__max_depth': 14, 'clf__class_weight': None}

[Fitting] HistGradientBoosting randomized search:
Fitting 3 folds for each of 8 candidates, totalling 24 fits
HGB best AUC (CV): 0.8501 | params: {'clf__max_iter': 100, 'clf__max_depth': 6, 'clf__learning_rate': 0.05}

[Comparison] Scoring candidates on validation split:
      Model  Val AUC  Val ACC
LR balanced 0.843605 0.774308
     LR raw 0.843006 0.801278
 HGB (best) 0.842193 0.804826
  RF (best) 0.832588 0.735273


## Threshold Calibration on Validation Set

In [72]:
# Select best model based on validation AUC
best_model = pipe_lr_bal

In [73]:
# Calibrate threshold by maximizing F1 on validation
best_model.fit(X_train, y_train)

val_proba = (
    best_model.predict_proba(X_val)[:, 1]
    if hasattr(best_model, 'predict_proba')
    else best_model.decision_function(X_val))

thresholds = np.linspace(0.2, 0.8, 61)
best_f1 = -1
best_thr = 0.5

for thr in thresholds:
    preds = (val_proba >= thr).astype(int)
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_val, preds, average='binary', zero_division=0)
    if f1 > best_f1:
        best_f1, best_thr, best_prec, best_rec = f1, thr, prec, rec

print(
    f"Best threshold on validation: {best_thr:.3f} | "
    f"F1={best_f1:.4f} | Prec={best_prec:.4f} | Rec={best_rec:.4f}",
    flush=True)

Best threshold on validation: 0.520 | F1=0.6474 | Prec=0.5703 | Rec=0.7487


## Final Training on Train+Val and One-Time Test Evaluation

In [74]:
X_trval = pd.concat([X_train, X_val], axis=0)
y_trval = pd.concat([y_train, y_val], axis=0)

final_model = best_model
final_model.fit(X_trval, y_trval)

# Test metrics (threshold-free AUC + thresholded Accuracy/Precision/Recall/F1)
test_proba = final_model.predict_proba(X_test)[:, 1] if hasattr(final_model, 'predict_proba') else final_model.decision_function(X_test)
test_auc = roc_auc_score(y_test, test_proba)

thr = best_thr
test_pred = (test_proba >= thr).astype(int)

test_acc = accuracy_score(y_test, test_pred)
prec, rec, f1, _ = precision_recall_fscore_support(y_test, test_pred, average='binary', zero_division=0)

print(f'Test Results: AUC={test_auc:.4f} | ACC={test_acc:.4f} | PREC={prec:.4f} | REC={rec:.4f} | F1={f1:.4f} | thr={thr:.3f}')

Test Results: AUC=0.8481 | ACC=0.7693 | PREC=0.5493 | REC=0.7299 | F1=0.6269 | thr=0.520
